In [1]:
pip install ultralytics torch torchvision opencv-python faiss-cpu numpy scipy easyocr

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Importing Modules

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from ultralytics import YOLO
from scipy.optimize import linear_sum_assignment
import json
from collections import deque
import easyocr
import logging
import time

In [ ]:
#   Configure logging  
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('player_tracking_debug.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# COSAM Module
class COSAM(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(in_channels, in_channels // reduction, 1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(in_channels // reduction, in_channels, 1, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        y = self.avg_pool(x)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y)
        return x * y

# Appearance Model with COSAM 
class AppearanceModelWithCOSAM(nn.Module):
    def __init__(self):
        super().__init__()
        backbone = models.resnet50(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(backbone.children())[:-2])
        self.cosam = COSAM(in_channels=2048)
        self.pool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        feats = self.feature_extractor(x)
        feats = self.cosam(feats)
        pooled = self.pool(feats).flatten(1)
        return F.normalize(pooled, p=2, dim=1)

# Initialize models globally for reuse
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
yolo_model = YOLO('best.pt')
appearance_model = AppearanceModelWithCOSAM().to(device).eval()
reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())

preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_cosam_embedding(roi, model, device):
    try:
        img = preprocess(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)).unsqueeze(0).to(device)
        emb = model(img)[0]
        return emb.cpu().detach().numpy()
    except Exception as e:
        logger.error(f"Embedding extraction failed: {str(e)}")
        return None

# Jersey Number Recognition with EasyOCR 
def recognize_jersey(roi):
    try:
        if roi is None or roi.size == 0:
            return None
        if len(roi.shape) == 2 or roi.shape[2] == 1:
            roi_rgb = cv2.cvtColor(roi, cv2.COLOR_GRAY2RGB)
        else:
            roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        h, w = roi_rgb.shape[:2]
        crop = roi_rgb[int(h*0.3):int(h*0.8), int(w*0.2):int(w*0.8)]
        results = reader.readtext(crop, detail=0)
        for text in results:
            cleaned = ''.join(filter(str.isdigit, text))
            if 1 <= len(cleaned) <= 2:
                return int(cleaned)
    except Exception as e:
        logger.error(f"Jersey recognition failed: {str(e)}")
    return None

# Enhanced Tracker 
class Track:
    def __init__(self, track_id, box, emb, roi):
        self.track_id = track_id
        self.box = box
        self.embeddings = [emb] if emb is not None else []
        self.rois = deque([roi], maxlen=4)
        self.hits = 1
        self.no_losses = 0
        self.global_id = None
        self.jersey = recognize_jersey(roi)
        logger.info(f"Created new track {track_id} with box {box}")
    
    def update(self, box, emb, roi):
        self.box = box
        if emb is not None:
            self.embeddings.append(emb)
        self.rois.append(roi)
        self.hits += 1
        self.no_losses = 0
        if self.jersey is None:
            self.jersey = recognize_jersey(roi)
        logger.info(f"Updated track {self.track_id}")
    
    def get_average_embedding(self):
        if not self.embeddings:
            return None
        return np.mean(self.embeddings, axis=0)

def iou(bb_test, bb_gt):
    try:
        xx1 = max(bb_test[0], bb_gt[0])
        yy1 = max(bb_test[1], bb_gt[1])
        xx2 = min(bb_test[2], bb_gt[2])
        yy2 = min(bb_test[3], bb_gt[3])
        w = max(0., xx2 - xx1)
        h = max(0., yy2 - yy1)
        intersection = w * h
        area_test = (bb_test[2] - bb_test[0]) * (bb_test[3] - bb_test[1])
        area_gt = (bb_gt[2] - bb_gt[0]) * (bb_gt[3] - bb_gt[1])
        return intersection / (area_test + area_gt - intersection + 1e-6)
    except Exception:
        return 0.0

def appearance_distance(emb1, emb2):
    if emb1 is None or emb2 is None:
        return 1.0
    try:
        return 1 - np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2) + 1e-6)
    except Exception:
        return 1.0

def assign_detections_to_tracks(tracks, detections):
    if not tracks or not detections:
        return [], list(range(len(detections))), list(range(len(tracks)))
    
    cost_matrix = np.zeros((len(tracks), len(detections)))
    for t, track in enumerate(tracks):
        for d, (box, emb, roi, jersey) in enumerate(detections):
            iou_score = iou(track.box, box)
            app_dist = appearance_distance(track.get_average_embedding(), emb)
            jersey_sim = 0.0 if (track.jersey is not None and jersey is not None and track.jersey == jersey) else 0.5
            cost_matrix[t, d] = 0.6 * app_dist + 0.2 * (1 - iou_score) + 0.2 * jersey_sim
    
    try:
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
    except Exception as e:
        logger.error(f"Assignment failed: {str(e)}")
        return [], list(range(len(detections))), list(range(len(tracks)))
    
    matches, unmatched_dets, unmatched_trks = [], [], []
    for t in range(len(tracks)):
        if t not in row_ind:
            unmatched_trks.append(t)
    for d in range(len(detections)):
        if d not in col_ind:
            unmatched_dets.append(d)
    
    for r, c in zip(row_ind, col_ind):
        if cost_matrix[r, c] < 0.7:
            matches.append((r, c))
        else:
            unmatched_trks.append(r)
            unmatched_dets.append(c)
    
    logger.info(f"Assignment: {len(matches)} matches, {len(unmatched_dets)} unmatched dets, {len(unmatched_trks)} unmatched tracks")
    return matches, unmatched_dets, unmatched_trks

#  Robust Video Processing 
def process_video(video_path, output_path):
    logger.info(f"Processing video: {video_path}")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logger.error(f"Failed to open video: {video_path}")
        return {}
    
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS) or 30
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    tracks = []
    next_id = 1
    max_age = 10
    frame_num = 0
    all_tracks = {}
    track_history = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_num += 1
        start_time = time.time()
        logger.info(f"Processing frame {frame_num}")
        
        # Detect players
        try:
            results = yolo_model(frame)
            detections = []
            if not results[0].boxes:
                logger.warning(f"No detections in frame {frame_num}")
            else:
                logger.info(f"Found {len(results[0].boxes)} detections")
                
            for i, box in enumerate(results[0].boxes):
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                # Validate bounding box
                if x2 <= x1 or y2 <= y1 or (x2 - x1) < 10 or (y2 - y1) < 10:
                    logger.warning(f"Invalid box: {x1},{y1},{x2},{y2}")
                    continue
                
                roi = frame[y1:y2, x1:x2]
                if roi.size == 0 or roi.shape[0] == 0 or roi.shape[1] == 0:
                    logger.warning(f"Empty ROI: {x1},{y1},{x2},{y2}")
                    continue
                
                emb = get_cosam_embedding(roi, appearance_model, device)
                if emb is None:
                    logger.warning(f"Embedding extraction failed for box {i}")
                    continue
                
                jersey = recognize_jersey(roi)
                detections.append(((x1, y1, x2, y2), emb, roi, jersey))
                logger.info(f"Detection {i}: box={x1},{y1},{x2},{y2} jersey={jersey}")
        except Exception as e:
            logger.error(f"Detection failed: {str(e)}")
            detections = []
        
        # Track management
        matches, unmatched_dets, unmatched_trks = assign_detections_to_tracks(tracks, detections)
        
        # Update matched tracks
        for t_idx, d_idx in matches:
            box, emb, roi, jersey = detections[d_idx]
            tracks[t_idx].update(box, emb, roi)
        
        # Create new tracks for unmatched detections
        for d_idx in unmatched_dets:
            box, emb, roi, jersey = detections[d_idx]
            new_track = Track(next_id, box, emb, roi)
            tracks.append(new_track)
            track_history[next_id] = new_track
            next_id += 1
        
        # Mark unmatched tracks
        for t_idx in unmatched_trks:
            tracks[t_idx].no_losses += 1
        
        # Remove stale tracks
        new_tracks = []
        for track in tracks:
            if track.no_losses <= max_age:
                new_tracks.append(track)
            else:
                logger.info(f"Removing track {track.track_id} after {track.no_losses} losses")
                # Save final state before removal
                all_tracks[track.track_id] = {
                    'embedding': track.get_average_embedding().tolist() if track.get_average_embedding() is not None else [],
                    'jersey': track.jersey
                }
        tracks = new_tracks
        
        # Draw and collect embeddings
        for track in tracks:
            x1, y1, x2, y2 = track.box
            color = (int(track.track_id * 50) % 256, int(track.track_id * 100) % 256, int(track.track_id * 150) % 256)
            label = f'ID {track.track_id}'
            if track.jersey is not None:
                label += f'|J{track.jersey}'
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        out.write(frame)
        logger.info(f"Frame processed in {time.time()-start_time:.2f}s")
    
    # Save remaining active tracks
    for track in tracks:
        all_tracks[track.track_id] = {
            'embedding': track.get_average_embedding().tolist() if track.get_average_embedding() is not None else [],
            'jersey': track.jersey
        }
    
    cap.release()
    out.release()
    logger.info(f"Video processing complete. Collected {len(all_tracks)} tracks")
    return all_tracks

#  Cross-Camera Mapping 
def cross_camera_mapping(tracks_a, tracks_b):
    if not tracks_a or not tracks_b:
        logger.error("One or both track collections are empty")
        return {}, {}, {}
    
    ids_a = list(tracks_a.keys())
    ids_b = list(tracks_b.keys())
    
    # Prepare embeddings
    emb_a, valid_ids_a = [], []
    for tid in ids_a:
        emb = tracks_a[tid].get('embedding')
        if emb and isinstance(emb, list) and len(emb) > 0:
            emb_a.append(emb)
            valid_ids_a.append(tid)
    
    emb_b, valid_ids_b = [], []
    for tid in ids_b:
        emb = tracks_b[tid].get('embedding')
        if emb and isinstance(emb, list) and len(emb) > 0:
            emb_b.append(emb)
            valid_ids_b.append(tid)
    
    if not emb_a or not emb_b:
        logger.error("No valid embeddings for matching")
        return {}, {}, {}
    
    emb_a = np.array(emb_a)
    emb_b = np.array(emb_b)
    
    # Build cost matrix
    cost_matrix = np.zeros((len(emb_a), len(emb_b)))
    for i, emb_i in enumerate(emb_a):
        for j, emb_j in enumerate(emb_b):
            app_sim = 1 - np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j) + 1e-6)
            jersey_sim = 0.0 if (
                tracks_a[valid_ids_a[i]].get('jersey') is not None and
                tracks_b[valid_ids_b[j]].get('jersey') is not None and
                tracks_a[valid_ids_a[i]]['jersey'] == tracks_b[valid_ids_b[j]]['jersey']
            ) else 0.5
            cost_matrix[i, j] = 0.8 * app_sim + 0.2 * jersey_sim
    
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    mapping = {}
    global_id = 1
    tacticam_to_global = {}
    broadcast_to_global = {}
    
    for i, j in zip(row_ind, col_ind):
        if cost_matrix[i, j] < 0.7:
            tid_a = valid_ids_a[i]
            tid_b = valid_ids_b[j]
            tacticam_to_global[tid_a] = global_id
            broadcast_to_global[tid_b] = global_id
            mapping[(tid_a, tid_b)] = global_id
            global_id += 1
    
    logger.info(f"Created {len(mapping)} cross-camera mappings")
    return mapping, tacticam_to_global, broadcast_to_global


c:\Users\risha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\risha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
#  Main Execution 
if __name__ == "__main__":
    logger.info("Starting player tracking system please wait 5-10 minutes for the generated video output and json file")
    logger.info("Loading YOLO model...")
    
    try:
        
        # Load tacticam video by changing the input file name path
        input_tacticam = 'tacticam.mp4' 
        logger.info("Processing TactiCam video...")
        
        tracks_tacticam = process_video(input_tacticam, 'output_tacticam.mp4')
        logger.info(f"TactiCam tracks: {len(tracks_tacticam)}")
        
        # Load tacticam video by changing the input file name path
        input_broadcast = 'broadcast.mp4'
        logger.info("Processing Broadcast video...")
        
        tracks_broadcast = process_video(input_broadcast, 'output_broadcast.mp4')
        logger.info(f"Broadcast tracks: {len(tracks_broadcast)}")
        
        logger.info("Cross-camera mapping...")
        mapping, tacticam_to_global, broadcast_to_global = cross_camera_mapping(
            tracks_tacticam, tracks_broadcast
        )
        
        with open('player_mapping.json', 'w') as f:
            json.dump({
                'mapping': {f"{k[0]}-{k[1]}": v for k, v in mapping.items()},
                'tacticam_to_global': tacticam_to_global,
                'broadcast_to_global': broadcast_to_global
            }, f, indent=2)
        
        logger.info(f"Player mapping complete. Saved {len(mapping)} mappings")
    except Exception as e:
        logger.exception("Critical error in main execution")

2025-06-26 16:01:41,288 - INFO - Starting player tracking system please wait 10-15 minutes for the first run
2025-06-26 16:01:41,291 - INFO - Loading YOLO model...
2025-06-26 16:01:41,292 - INFO - Processing TactiCam video...
2025-06-26 16:01:41,292 - INFO - Processing video: tacticam.mp4
2025-06-26 16:01:41,410 - INFO - Processing frame 1



0: 384x640 22 players, 2 referees, 101.4ms
Speed: 12.3ms preprocess, 101.4ms inference, 176.8ms postprocess per image at shape (1, 3, 384, 640)


2025-06-26 16:01:44,204 - INFO - Found 24 detections
2025-06-26 16:01:44,387 - INFO - Detection 0: box=314,526,366,590 jersey=None
2025-06-26 16:01:44,410 - INFO - Detection 1: box=1716,320,1748,383 jersey=None
2025-06-26 16:01:44,434 - INFO - Detection 2: box=1133,480,1175,543 jersey=None
2025-06-26 16:01:44,459 - INFO - Detection 3: box=928,197,967,247 jersey=None
2025-06-26 16:01:44,511 - INFO - Detection 4: box=901,573,942,639 jersey=None
2025-06-26 16:01:44,539 - INFO - Detection 5: box=1131,344,1168,403 jersey=None
2025-06-26 16:01:44,563 - INFO - Detection 6: box=1659,461,1695,524 jersey=None
2025-06-26 16:01:44,587 - INFO - Detection 7: box=1298,810,1329,889 jersey=None
2025-06-26 16:01:44,615 - INFO - Detection 8: box=361,377,391,433 jersey=None
2025-06-26 16:01:44,640 - INFO - Detection 9: box=342,804,377,878 jersey=None
2025-06-26 16:01:44,663 - INFO - Detection 10: box=289,446,316,513 jersey=None
2025-06-26 16:01:44,688 - INFO - Detection 11: box=734,346,771,403 jersey=None


0: 384x640 22 players, 1 referee, 74.8ms
Speed: 2.9ms preprocess, 74.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


2025-06-26 16:01:45,281 - INFO - Found 23 detections
2025-06-26 16:01:45,303 - INFO - Detection 0: box=900,575,932,639 jersey=None
2025-06-26 16:01:45,327 - INFO - Detection 1: box=315,526,365,591 jersey=None
2025-06-26 16:01:45,352 - INFO - Detection 2: box=768,392,802,456 jersey=None
2025-06-26 16:01:45,375 - INFO - Detection 3: box=361,376,395,432 jersey=None
2025-06-26 16:01:45,397 - INFO - Detection 4: box=1712,320,1750,384 jersey=None
2025-06-26 16:01:45,421 - INFO - Detection 5: box=445,209,480,265 jersey=None
2025-06-26 16:01:45,441 - INFO - Detection 6: box=1133,479,1173,543 jersey=None
2025-06-26 16:01:45,466 - INFO - Detection 7: box=929,199,964,246 jersey=None
2025-06-26 16:01:45,489 - INFO - Detection 8: box=1130,345,1168,403 jersey=None
2025-06-26 16:01:45,515 - INFO - Detection 9: box=1660,461,1698,524 jersey=None
2025-06-26 16:01:45,542 - INFO - Detection 10: box=732,346,771,402 jersey=None
2025-06-26 16:01:45,568 - INFO - Detection 11: box=289,446,316,512 jersey=None
2

...

2025-06-26 16:06:45,106 - WARNING - No detections in frame 130
2025-06-26 16:06:45,141 - INFO - Frame processed in 0.12s
2025-06-26 16:06:45,148 - INFO - Processing frame 131



0: 384x640 (no detections), 75.2ms
Speed: 2.3ms preprocess, 75.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)


2025-06-26 16:06:45,231 - WARNING - No detections in frame 131
2025-06-26 16:06:45,264 - INFO - Frame processed in 0.12s
2025-06-26 16:06:45,273 - INFO - Processing frame 132



0: 384x640 (no detections), 75.3ms
Speed: 1.8ms preprocess, 75.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


2025-06-26 16:06:45,355 - WARNING - No detections in frame 132
2025-06-26 16:06:45,381 - INFO - Frame processed in 0.11s
2025-06-26 16:06:45,408 - INFO - Video processing complete. Collected 18 tracks
2025-06-26 16:06:45,422 - INFO - Broadcast tracks: 18
2025-06-26 16:06:45,423 - INFO - Cross-camera mapping...
2025-06-26 16:06:45,436 - INFO - Created 18 cross-camera mappings
2025-06-26 16:06:45,439 - INFO - Player mapping complete. Saved 18 mappings
